In [1]:
from utils import RMSHisteresisEstimator

from sklearn.metrics  import accuracy_score
import numpy as np
import tqdm
from sklearn.metrics import classification_report
import torch
import joblib

from utils import custom_metric_torch_batch

dataset_root_path = 'D:\Skoltech\Thesis\Calculations/time-series-analysis\ML_1/dataset'

In [2]:
rmshist = RMSHisteresisEstimator()



In [3]:
data_test = joblib.load(dataset_root_path + "/test/data-multi-test-fixed.joblib")
labels_test = joblib.load(dataset_root_path + "/test/labels-multi-test-fixed.joblib")


data_test = np.array(data_test)
labels_test = np.array(labels_test)

In [4]:
data_test[0]

array([533345., 567678., 598512., ..., 524497., 551112., 574330.])

In [5]:
multi_preds = []
binary_predictions = []
binary_true_labels = []


for unit, label in tqdm.tqdm(zip(data_test, labels_test), total=len(data_test)):

    binary_true_labels.append(label) #forgot to sum it up
    pred = rmshist.predict(
        time_series=unit/ 1e3,
        activation_current=500,
        deactivation_coef=0.95,
        )
    
    multi_preds.append(pred)
    binary_predictions.append(pred.sum() > 0)

binary_true_labels = [x.sum()>0 for x in binary_true_labels] # sum it up here


100%|██████████| 1971/1971 [02:07<00:00, 15.49it/s]


In [6]:
acc = accuracy_score(
                                                    y_pred = binary_predictions, 
                                                    y_true = binary_true_labels)
print(f"Accuracy on binary classification: { acc * 100:.3f}%\n")
print(classification_report(y_pred = binary_predictions, 
                                                    y_true = binary_true_labels, digits=4))

Accuracy on binary classification: 42.009%

              precision    recall  f1-score   support

       False     1.0000    0.0322    0.0623      1181
        True     0.4087    1.0000    0.5802       790

    accuracy                         0.4201      1971
   macro avg     0.7043    0.5161    0.3213      1971
weighted avg     0.7630    0.4201    0.2699      1971



In [7]:
y_true_multi_torch = torch.tensor(labels_test, dtype=torch.float).unsqueeze(0)
print(y_true_multi_torch.shape)

multi_preds = np.array(multi_preds)
y_pred_multi_torch = torch.tensor(multi_preds, dtype=torch.uint8).unsqueeze(0)
print(y_pred_multi_torch.shape)



print(f'Custom Multilabel Score: {custom_metric_torch_batch(y_pred=y_pred_multi_torch, y_true=y_true_multi_torch).mean()*100:.4f}%')

torch.Size([1, 1971, 5601])
torch.Size([1, 1971, 5601])
Custom Multilabel Score: 24.3262%
